In [1]:
import pandas as pd
import numpy as np
import requests
import os 
import json
from bs4 import BeautifulSoup
import collections
collections.Callable = collections.abc.Callable

In [2]:
useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

In [3]:
propublica_token = os.environ['propub_token']

In [17]:
root = "https://api.propublica.org"
congress = '117'
chamber = "house" 
endpoint = "/congress/v1/{congress}/{chamber}/members.json".format(congress = congress,
                                                                   chamber = chamber)

In [5]:
headers = {'X-API-Key': propublica_token,
                     'User-agent': useragent,
                     'From': 'nqj5zk@virginia.edu'} # Contact me

# Goal: Get Bills Sponsored by Bob Good

## 1. Get Bob Good's ID Number 

In [18]:
r = requests.get(root + endpoint,
                headers = headers)
myjson = json.loads(r.text)
r

<Response [200]>

In [7]:
membersdf = pd.json_normalize(myjson, record_path = 
                              ['results', 'members'])
membersdf.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,...,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,...,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,...,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,...,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09


In [8]:
bobgood=membersdf.query("last_name == 'Good'")
bobgood

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,...,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
158,G000595,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,None,Good,None,1965-09-11,M,...,1213 Longworth House Office Building,202-225-4711,None,VA,5,False,5105,4.11,77.32,22.56


In [9]:
bobgoodid = bobgood.reset_index()['id'][0]
bobgoodid

'G000595'

## Step 2: Use Bob Good's ID to Query Bills API

In [28]:
root = "https://api.propublica.org"
endpoint = "/congress/v1/members/{memberid}/bills/{billtype}/.json".format(memberid = bobgoodid,
                                                                           billtype = 'introduced')

r = requests.get(root + endpoint,
               headers = headers)

myjson = json.loads(r.text)
myjson

#bgbills1 = pd.json_normalize(myjson, record_path = 
                                        # ['results', 'bills'])

{'status': '404', 'error': 'Not Found'}

In [30]:
r2 = requests.get(root + endpoint,
               headers = headers,
               params = {'offset':20})

myjson2 = json.loads(r2.text)
#bgbills1 = pd.json_normalize(myjson2, record_path = ['results', 'bills'])

{'status': '404', 'error': 'Not Found'}

In [ ]:
bgbills = pd.concat[bgbills1, bgbills2], ignore_index=True
bgbills.head(3).T

In [ ]:
urltoscrape = bgbills['congressdotgov_url'][11]

In [ ]:
r = requests.get(urltoscrape,
                headers = {'User-Agent':useragent,
                          'From': 'nqj5zk@virginia.edu'})
myhtml = BeautifulSoup(r.text, 'html.parser')

In [ ]:
#Find all finds all of the paragraphs on the website
myhtml.find_all('p', "lbexIndent", style="text-align:left")[0].text